*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-02-09 07:36:16--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip      [               <=>  ]  24.88M  2.15MB/s    in 11s     

2021-02-09 07:36:28 (2.19 MB/s) - ‘book-crossings.zip’ saved [26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
print(df_books.head(3))
print(df_ratings.head(3))

         isbn                 title                author
0  0195153448   Classical Mythology    Mark P. O. Morford
1  0002005018          Clara Callan  Richard Bruce Wright
2  0060973129  Decision in Normandy          Carlo D'Este
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0


In [6]:
books_with_ratings = pd.merge(df_ratings,df_books,on='isbn')
books_with_ratings.head()

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,2313,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose
2,6543,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
3,8680,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose
4,10314,034545104X,9.0,Flesh Tones: A Novel,M. J. Rose


In [8]:
user_RatingCount = books_with_ratings.groupby('user')['rating'].count().reset_index().rename(columns = {'rating':'userTotalRatingCount'})
book_RatingCount = books_with_ratings.groupby('title')['rating'].count().reset_index().rename(columns = {'rating':'bookTotalRatingCount'})
books_with_ratings = books_with_ratings.merge(user_RatingCount,how='left', left_on='user', right_on='user')
books_with_ratings = books_with_ratings.merge(book_RatingCount, how='left', left_on='title', right_on='title')

In [9]:
books_with_ratings.head()

,user,isbn,rating,title,author,userTotalRatingCount,bookTotalRatingCount
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,1,60
1,2313,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose,36,60
2,6543,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,425,60
3,8680,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose,25,60
4,10314,034545104X,9.0,Flesh Tones: A Novel,M. J. Rose,167,60


In [10]:
books_with_ratings_2 = books_with_ratings.loc[(books_with_ratings['userTotalRatingCount']>=200) & (books_with_ratings['bookTotalRatingCount']>=100)]

In [12]:
print(books_with_ratings_2.head())
books_with_ratings_3 = books_with_ratings_2.drop_duplicates(['title', 'user'])
books_with_ratings_3.head()

      user        isbn  ...  userTotalRatingCount bookTotalRatingCount
63  278418  0446520802  ...                  3996                  650
65    3363  0446520802  ...                   890                  650
66    7158  0446520802  ...                   398                  650
69   11676  0446520802  ...                 11144                  650
74   23768  0446520802  ...                  1690                  650

[5 rows x 7 columns]


,user,isbn,rating,title,author,userTotalRatingCount,bookTotalRatingCount
63,278418,0446520802,0.0,The Notebook,Nicholas Sparks,3996,650
65,3363,0446520802,0.0,The Notebook,Nicholas Sparks,890,650
66,7158,0446520802,10.0,The Notebook,Nicholas Sparks,398,650
69,11676,0446520802,10.0,The Notebook,Nicholas Sparks,11144,650
74,23768,0446520802,6.0,The Notebook,Nicholas Sparks,1690,650


In [ ]:
# Convert to 2D matrıx
books_with_ratings_pivot = books_with_ratings_3.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
books_with_ratings_matrix = csr_matrix(books_with_ratings_pivot.values)

In [33]:
#[‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]
model_knn = NearestNeighbors(algorithm='brute', metric='cosine')
model_knn.fit(books_with_ratings_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [24]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  X = books_with_ratings_pivot[books_with_ratings_pivot.index == book]
  X = X.to_numpy().reshape(1,-1)
  distances, indices = model_knn.kneighbors(X,n_neighbors=5)
  recommended_books = []
  for i in range(0, len(distances.flatten())):
    if i == 0:
      recommended_books.append(books_with_ratings_pivot.index[indices.flatten()[0]])
      recommended_books.append([])
    else:
      recommended_books[1].append([books_with_ratings_pivot.index[indices.flatten()[i]], distances.flatten()[i]])

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [35]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Blue Diary', 0.7145698], ['The Lovely Bones: A Novel', 0.7158567], ['I Know This Much Is True', 0.7485672], ['Icy Sparks', 0.7646523]]]
